In [1]:
import numpy as np
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider

In [2]:
from prev_ob_models.Birgiolas2020.isolated_cells import MC

In [3]:
mc = MC()

In [4]:
from neuron import h

vec = h.Vector()
vec.record(mc.cell.soma(0.5)._ref_v, 0.5)


h.tstop = 10000

#h.newPlotV()

In [5]:
ic = h.IClamp(mc.cell.soma(0.5))
ic2 = h.IClamp(mc.cell.soma(0.5))
ic3 = h.IClamp(mc.cell.soma(0.5))

In [6]:
ic.delay = 1000
ic.dur = 2000
ic.amp = -0.1


ic2.delay = 4000
ic2.dur = 2000
ic2.amp = 0.111


ic3.delay = 7000
ic3.dur = 2000
ic3.amp = 0.111*2

In [7]:
params = [    
    { "start": 34.77,  "attr": "Ra",        "low": 5.0, "high": 100.0, "lists": ["all"] },
    { "start": 2.706,   "attr": "cm",        "low": 0.1, "high": 4.0, "lists": ["all"] },
    { "start": 49.95,  "attr": "ena",       "low": 40.0, "high": 50.0, "lists": ["all"] },
    { "start": -70.03,  "attr": "ek",        "low": -100.0, "high": -70.0, "lists": ["all"] },
    { "start": -64.42,  "attr": "e_pas",     "low": -70.0, "high": -50.0, "lists": ["all"] },
    { "start": 0.0005955, "attr": "g_pas",     "low": 0, "high": 0.00003, "lists": ["all"] },    
    { "start": 0.5955,  "attr": "sh_Na",     "low": 0, "high": 10, "lists": ["all"] },    
    { "start": 10,  "attr": "tau_CaPool",     "low": 1, "high": 500, "lists": ["all"] },    

    { "start":  0.87485,  "attr": "gbar_Na",     "low": 0, "high": 0.05, "lists": ["all"] },
    { "start": 0.0297,  "attr": "gbar_Kd",     "low": 0, "high": 0.04, "lists": ["all"] },
    { "start": 0.000264,  "attr": "gbar_Kslow",  "low": 0, "high": 0.004, "lists": ["all"] },
    { "start": 0.07215,  "attr": "gbar_KA",     "low": 0, "high": 0.005, "lists": ["all"] },
    { "start": 0.001,  "attr": "gbar_KCa",    "low": 0, "high": 0.004, "lists": ["all"] },
    { "start": 0.00081441,  "attr": "gbar_LCa",  "low": 0, "high": 0.001, "lists": ["all"] },
    
    { "start": -30.805,  "attr": "eh",       "low": -40.0, "high": -25.0, "lists": ["apical"] },
    { "start": 0.00335,  "attr": "gbar_Ih",    "low": 0, "high": 0.00003, "lists": ["apical"] },
    { "start": 0.000107,  "attr": "gbar_CaT",    "low": 0, "high": 18e-3, "lists": ["apical"] },
]

In [8]:
best = [55.1610864800591,
 1.2011223808544307,
 45.858152596044036,
 -89.54201113602845,
 -65.69659324803041,
 1.373134023516298e-05,
 0.4579888639715568,
 2.203808438053512,
 0.02289944319857784,
 0.02183966039456448,
 0.0018319554558862273,
 0.0022899443198577843,
 0.0021340101963085055,
 0.0026953214448517935,
 -36.92224642477413,
 0.00010991732735317362,
 0.005630666540928916]

best = [36.733224907230976,
 1.218757975793375,
 42.86861019434199,
 -75.90029648025586,
 -64.26277961131602,
 1.2947334242010801e-05,
 0.29153326073896313,
 206.14023477532092,
 0.014343050971709938,
 0.022928918544327836,
 0.00020464235583769514,
 0.0029193616564435116,
 0.001147444077736795,
 6.314491456148351e-05,
 -35.69708470848702,
 5.9204949172227345e-06,
 0.0051634983498155766]

best = [13.021558319332806,
 0.1421244817119445,
 46.8584157330876,
 -93.26652964252163,
 -56.87131277268493,
 2.4402946120574333e-05,
 4.58994085936241,
 12.492102024675049,
 0.0137369123653096,
 0.012076032916784003,
 0.001009786028927451,
 0.001969382122174609,
 0.0006458839218994715,
 0.0007132677073125959,
 -29.718414466812085,
 1.1532769221682631e-05,
 0.0038349927360426094]

for i, p in enumerate(params):
    params[i]["start"] = best[i]

In [9]:
kwargs = {}
for pi, pv in enumerate(params):
    #print(pv["attr"])
    kwargs[pv["attr"]+"|"+str(pv["lists"])] = FloatSlider(
        value=pv["start"],
        min=pv["low"],
        max=pv["high"],
        step=(pv["high"]-pv["low"])/100.0,
        continuous_update=False,
        readout_format='.9f',
        description=(pv["attr"]+"|"+str(pv["lists"][0])).replace("gbar_","g")
    )
#kwargs

In [10]:
def evaluate(**kwargs):   
    param_keys = kwargs.keys()
    
    for key in param_keys:
        attr = key.split("|")[0]
        lists = eval(key.split("|")[1])
        
        for param_list in lists:
            for sec in getattr(mc.cell, param_list):
                if attr == "tau_CaPool":
                    setattr(h, attr, kwargs[key])
                else:
                    setattr(sec, attr, kwargs[key])
        
    h.run()
    
    from matplotlib import pyplot as plt
    plt.plot(vec.as_numpy())
    plt.show()
    
    return mc.cell.soma(0.5).v

In [11]:
interact(evaluate, **kwargs);

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTQ2Ljg1ODQxNTczMzA4NzYsIGNvbnRpbnVvdXNfdXBkYXRlPUZhbHNlLCBkZXNjcmlwdGlvbj11J2VuYXxhbGwnLCBtYXjigKY=


In [12]:
from neuron import gui